<p><b>Подключение библиотек и функций</b></p>

In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [27]:
def getSE(sigma, n):
    return sigma / np.sqrt(n)

def getSE2(sigma1, n1, sigma2, n2):
    return np.sqrt(sigma1**2 / n1 + sigma2**2 / n2)

def getZ(x, mu, se):
    return (x - mu) / se

def getPrByX(x, mu, se):
    return stats.norm.cdf(x, loc=mu, scale=se)

def getPrByZ(z):
    return stats.norm.cdf(z, loc=0, scale=1)

def getZbyPr2Tail(pr):
    return stats.norm.ppf((1-pr)/2, loc=0, scale=1)

def getZbyPr1Tail(pr):
    return stats.norm.ppf(1-pr, loc=0, scale=1)

def getPValue1Tail_Norm(z):
    return 1 - stats.norm.cdf(z, loc=0, scale=1)

def getPValue2Tail_Norm(z):
    return 2 * (stats.norm.cdf(-abs(z), loc=0, scale=1))

def getPValue2Tail_T(t, df):
    return 2 * stats.t.cdf(-abs(t), df=df)

def getPValue1Tail_T(t, df):
    return 1 - stats.t.cdf(t, df=df)

<p>Создание генеральных совокупностей</p>

In [3]:
mu1 = 70
sigma1 = 30
population1 = stats.norm.rvs(size=10000, loc=mu1, scale=sigma1)

In [4]:
mu2 = 60
sigma2 = 20
population2 = stats.norm.rvs(size=10000, loc=mu2, scale=sigma2)

In [5]:
mu3 = 2
sigma3 = 2
diff = stats.norm.rvs(size=10000, loc=mu3, scale=sigma3)
population3 = list(zip(population1, population1 + diff))

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [6]:
n = 40
sample_2015 = np.random.choice(population1, n, replace=False)
sample_2015

array([105.53136754,  79.6907848 ,  69.90648131,  86.71255662,
        87.04244403,  73.75280147,  83.79178721,  56.65395523,
        41.18353851,  73.51169843, -32.63284905,  93.54135332,
        77.74396736,  24.89972895,  58.02862815,  45.4235892 ,
       109.90735321,  46.69052225,  56.13297526,  81.46913807,
        85.59099142,  33.79635544,  58.34804135, 117.48329447,
        72.37851504,  98.8834888 ,  72.09533255,  47.65162535,
        56.16547511, 142.49939931,  81.07939422, 142.19378089,
        54.72770264, 153.09031315,  85.97995757,  64.06494566,
        53.81614448,  90.24388023,  99.71813754,  80.03558397])

<p>Статистики выборки</p>

In [7]:
#Среднее значение
x_bar = sample_2015.mean()
print(x_bar)

#Выборочная дисперсия
var_2015 = sample_2015.var()
print(var_2015)

#Стандартное отклонение (несмещенное)
s_2015 = sample_2015.std()
print(s_2015)

75.22060452636988
1090.0884031894052
33.01648683899311


<p><b><i>Доверительный интервал</i></b></p>

In [8]:
#Уровни доверия: 95%, 90%, 80%
prs = np.array([0.95, 0.9, 0.8])

In [12]:
#Стандартная ошибка
se = getSE(np.sqrt(var_2015),len(sample_2015))
se

5.220364937409561

In [42]:
#Доверительный интервал
lowers, uppers = [], []
for pr in prs:
    lowers.append(sample_2015.mean()+getZbyPr2Tail(pr)*se)
    uppers.append(sample_2015.mean()-getZbyPr2Tail(pr)*se)
for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers[i], uppers[i]))

Уровень доверия 95%: (64.988877, 85.452332)
Уровень доверия 90%: (66.633868, 83.807341)
Уровень доверия 80%: (68.530438, 81.910771)


In [44]:
#Доверительный интервал для t-распределения (при небольших n)
lowers_t, uppers_t = [], []
for pr in prs:
    t = stats.t.ppf((1-pr)/2,len(sample_2015)-1)
    lowers_t.append(sample_2015.mean()+t*se)
    uppers_t.append(sample_2015.mean()-t*se)

print("t-распределение")
for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers_t[i], uppers_t[i]))

t-распределение
Уровень доверия 95%: (64.661420, 85.779789)
Уровень доверия 90%: (66.424942, 84.016268)
Уровень доверия 80%: (68.415135, 82.026074)


<p>или</p>

In [48]:
#z-значения для двустороннего интервала
zs = [abs(getZbyPr2Tail(pr)) for pr in prs]
zs

[1.959963984540054, 1.6448536269514729, 1.2815515655446004]

In [49]:
#Предельная ошибка
delta = [z*se for z in zs]
delta

[10.231727263478433, 8.586736201308414, 6.690166858251362]

In [51]:
#Доверительный интервал
lowers = []
uppers = []
for d in delta:
    lowers.append(sample_2015.mean()-d)
    uppers.append(sample_2015.mean()+d)

for i in range(len(prs)):
    print("Уровень доверия %i%%: (%f, %f)" % (prs[i]*100, lowers[i], uppers[i]))

Уровень доверия 95%: (64.988877, 85.452332)
Уровень доверия 90%: (66.633868, 83.807341)
Уровень доверия 80%: (68.530438, 81.910771)


<p>$$H_0: \mu_{2014}=\mu_{2015}$$<p>
<p>$$H_A: \mu_{2014} \neq \mu_{2015}$$<p>

<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [55]:
mu_2014 = 65
n = 40
alpha = 0.05 #уровень значимости
sample_2015 = np.random.choice(population1, n, replace=False)
sample_2015

array([ 44.45990163, -11.1153643 ,  58.07514852,  53.96205214,
        17.21181976,  35.5511113 ,  80.72041478,  65.41519704,
        68.96150403,  78.76541244,   5.66071291, 119.1263839 ,
        91.29520994,  63.49225166,  29.57302202,  98.13573426,
        74.06048554,  44.74613237,  78.10628305,  76.45461862,
        37.23024965,  58.96913942,  63.97413949,  89.38229221,
        75.03304709,  56.41913727,  70.81799124,   1.45739406,
        57.84605828, 104.34290773,  48.00360972,  56.0228305 ,
       113.9452764 ,  67.11419237,  95.56636439,  50.10764455,
        61.92474549,  33.74812166,  74.48697327,  72.97783511])

In [65]:
#Среднее значение
x_bar_2015 = sample_2015.mean()
print(x_bar)

#Стандартное отклонение (несмещенное)
s_2015 = sample_2015.std()
print(s_2015)

#Стандартная ошибка
se = getSE(sample_2015.std(),len(sample_2015))
print(se)

75.22060452636988
28.33681471885317
4.480443807287994


<p><b>Проверка гипотезы</b></p>

In [66]:
#Z-score
z = getZ(x_bar_2015,mu_2014,se)
z

-0.7698568735114742

In [68]:
#P-Value
pvalue = getPValue2Tail_Norm(z)
pvalue

0.44138479849514234

In [69]:
#P-Value для t-распределения
df = n - 1 #степень свободы
se = getSE(sample_2015.std(ddof=1),n)
t = getZ(sample_2015.mean(),mu_2014,se)
pvalue_t = getPValue2Tail_T(t,df)
pvalue_t

0.45172259119713765

<p>или</p>

In [71]:
tvalue, pvalue_t = stats.ttest_1samp(sample_2015, mu_2014)
tvalue,pvalue_t

(-0.7601727537543952, 0.45172259119713765)

In [72]:
pvalue_t = 2 * stats.t.cdf(-abs(tvalue),df)
pvalue_t

0.45172259119713765

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [73]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

alpha <= p-value
Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)


<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [76]:
alpha = 0.05 #уровень значимости

In [77]:
import random

n_items = 40
sample_items = np.array(random.sample(population3, n_items))
sample_items

array([[ 84.00469378,  89.05698686],
       [ 82.16476959,  81.59722861],
       [ 43.72106622,  42.08918964],
       [ 76.98227604,  80.78891458],
       [ 71.28291116,  72.81315891],
       [ 44.01000132,  45.68450062],
       [ 27.13551175,  28.05682816],
       [ 25.88728551,  30.78651255],
       [ 72.01556902,  73.91481088],
       [ 91.59397348,  92.0319972 ],
       [ 79.71095479,  81.639429  ],
       [ 41.11876423,  42.94204021],
       [134.35339295, 133.56396672],
       [ 68.98646689,  68.60277915],
       [ 78.57310761,  84.35569169],
       [ 52.60269626,  56.61047215],
       [ 30.96561848,  34.00414567],
       [ 79.84283045,  85.52142713],
       [ 50.10168833,  51.53911163],
       [ 76.78766759,  81.16724056],
       [ 61.19138594,  64.88261573],
       [ 81.64324495,  85.61582284],
       [ 92.8836649 ,  96.11502888],
       [105.7525099 , 107.5803945 ],
       [114.83394097, 119.0766586 ],
       [ 81.56875376,  82.33828157],
       [102.23523026, 103.50276144],
 

In [78]:
sample_diff = sample_items[:,0] - sample_items[:,1]
sample_diff

array([-5.05229308,  0.56754097,  1.63187658, -3.80663854, -1.53024775,
       -1.6744993 , -0.92131641, -4.89922704, -1.89924186, -0.43802372,
       -1.92847421, -1.82327598,  0.78942623,  0.38368774, -5.78258408,
       -4.00777589, -3.03852719, -5.67859668, -1.4374233 , -4.37957297,
       -3.69122979, -3.97257789, -3.23136399, -1.8278846 , -4.24271763,
       -0.76952781, -1.26753119, -1.22959697, -0.50336302,  1.08838133,
        2.0746479 , -1.04888247, -1.33763844, -2.01483593, -0.59151963,
       -0.02367128, -1.68453787,  2.93014855, -1.44546158, -1.72617477])

In [79]:
#Среднее значение
x_bar_diff = sample_diff.mean()
print("x_bar_diff =", x_bar_diff)

#Стандартное отклонение (несмещенное)
s_diff = sample_diff.std()
print("s_diff =", s_diff)

#Стандартная ошибка
se_diff = getSE(sample_diff.std(),len(sample_diff))
print("se_diff =", se_diff)

x_bar_diff = -1.7360130890297856
s_diff = 2.0659293499867992
se_diff = 0.3266521115474718


<p><b>Проверка гипотезы</b></p>

<p>$$H_0: \mu_{diff}=0$$<p>
<p>$$H_A: \mu_{diff} \neq 0$$<p>

In [80]:
#Z-score
z = getZ(x_bar_diff,0,se)
z

-0.38259057603151797

In [81]:
#P-Value
pvalue = getPValue2Tail_Norm(z)
pvalue

0.7020233588238028

In [84]:
#P-Value для t-распределения
df = len(sample_diff)-1
se = getSE(sample_diff.std(ddof=1),n)
t = getZ(x_bar_diff,0,se)
pvalue_t = getPValue2Tail_T(t,df)
t,pvalue_t

(-5.247710075978013, 5.716073144810502e-06)

<p>или</p>

In [83]:
tvalue, pvalue_t = stats.ttest_rel(sample_items[:,0], sample_items[:,1])
print("t-value =", tvalue)
print("p-value =", pvalue_t)

t-value = -5.247710075978012
p-value = 5.7160731448105235e-06


In [85]:
pvalue_t = 2 * stats.t.cdf(-abs(tvalue), df)
pvalue_t

5.7160731448105235e-06

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [86]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

alpha <= p-value
Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)


<p><b><i>Выборка</i></b></p>

<p>Исходные данные</p>

In [88]:
alpha = 0.05 #уровень значимости

In [89]:
n_london = 40
sample_london = np.random.choice(population1, n_london, replace=False)
sample_london

array([ 43.33707286,  34.59966124,  27.12876111,  80.79093155,
        77.80085673,   9.52306341,  86.66671683,  72.14438563,
        27.60525757,  79.8023942 ,  51.51526747,  22.21580893,
        23.05534917,  34.01714433,  64.54766163,  46.91357504,
        96.74829709,  61.27882118,  59.18551451,  30.11371152,
       122.11152213,  39.33634689,  64.18163241, 124.93446828,
        47.45328883,  11.23074822, 101.41208139, 114.83394097,
       155.47085484,  63.28324226,  77.32124117,  55.66603751,
       110.99229553,  41.9474302 ,  56.91743259,  93.09940236,
        80.61763703,  56.63629335, 114.8987772 , 107.89081446])

In [90]:
n_moscow = 50
sample_moscow = np.random.choice(population2, n_moscow, replace=False)
sample_moscow

array([ 50.27703975,  58.43525331,  47.36494725,  75.61781085,
        41.60511586,  86.17477432,  40.25103117,  31.85992475,
        24.32574237,  80.32940007,  62.55529613,  60.63903666,
        42.67909038,  54.37931907,  55.57846784,  48.58970482,
        38.91968427,  82.45773376,  97.9026165 ,  60.69468135,
        68.21859493,  34.78841535,  39.28649043,  47.08257295,
        53.44553724,  42.46347022,  36.80542551,  87.52948999,
        78.79294172,  57.09148128,  42.24055101,  96.22293787,
        86.74882771,  72.84187376,  48.86457974,  80.98828461,
        35.30382306,  46.03146857, 107.30262423,  75.69479656,
        60.77911391,  47.98184941,  40.2107153 ,  77.24420587,
        43.85146687,  29.16359402,  40.32942005,  28.98758864,
        55.2523416 ,  73.45802922])

In [91]:
#Среднее значение
x_bar_london = sample_london.mean()
x_bar_moscow = sample_moscow.mean()
print("x_bar_london =", x_bar_london)
print("x_bar_moscow =", x_bar_moscow)

#Стандартное отклонение (несмещенное)
s_london = sample_london.std()
s_moscow = sample_moscow.std()
print("s_london =", s_london)
print("s_moscow =", s_moscow)

#Стандартная ошибка
se_london_moscow = getSE2(s_london,len(sample_london),s_moscow,len(sample_moscow))
print("se_london_moscow =", se_london_moscow)

x_bar_london = 66.73064349015172
x_bar_moscow = 57.51278364254423
s_london = 34.19124556120958
s_moscow = 20.111223200185716
se_london_moscow = 6.108621595566966


<p><b>Проверка гипотезы</b></p>

<p>$$H_0: \mu_{treated}-\mu_{control}=0$$<p>
<p>$$H_A: \mu_{treated}-\mu_{control} \neq 0$$<p>

In [92]:
#Z-score
z = getZ(x_bar_london-x_bar_moscow,0,se_london_moscow)
z

1.508991791912091

In [93]:
#P-Value
pvalue = getPValue2Tail_Norm(z)
pvalue

0.13130088038471968

In [148]:
#P-Value для t-распределения
df = 0
se = getSE2(sample_london.std(ddof=1),len(sample_london),sample_moscow.std(ddof=1),len(sample_moscow))
t = getZ(x_bar_london-x_bar_moscow,0,se)
if len(sample_london) < len(sample_moscow):
    df = len(sample_moscow) - 1
else:
    df = len(sample_london) - 1


pvalue_t = getPValue2Tail_T(t,df) 
t,pvalue_t

(1.4908346848386829, 0.1424147898069532)

<p>или</p>

In [151]:
tvalue, pvalue_t = stats.ttest_ind(sample_london,sample_moscow,equal_var=False)
tvalue,pvalue_t

(1.4908346848386826, 0.14125899962960098)

<p>или</p>

In [152]:
tvalue, pvalue_t =  stats.ttest_ind_from_stats(x_bar_london, sample_london.std(ddof=1), n_london, x_bar_moscow, sample_moscow.std(ddof=1), n_moscow, equal_var=False)
pvalue_t

0.14125899962960098

In [143]:
pvalue_t = 2 * stats.t.cdf(-abs(tvalue), df)
pvalue_t

0.1424147898069532

<p><b>Проверка гипотезы на соответствие уровню значимости $\alpha$</b></p>

In [153]:
if alpha > pvalue:
    print("alpha > p-value")
    print("Отказываемся от нулевую гипотезы в пользу альтернативной")
else:
    print("alpha <= p-value")
    print("Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)")

alpha <= p-value
Принимаем нулевую гипотезу (не отказываемся от нулевой гипотезы)
